In [4]:
import numpy as np

def zigzag_scan(matrix):
    Y = np.array(matrix[0], dtype=np.float64)
    Cr = np.array(matrix[1], dtype=np.float64)
    Cb = np.array(matrix[2], dtype=np.float64)

    Y = np.concatenate([np.diagonal(Y[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - Y.shape[0], Y.shape[0])])
    Cr = np.concatenate([np.diagonal(Cr[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - Cr.shape[0], Cr.shape[0])])
    Cb = np.concatenate([np.diagonal(Cb[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - Cb.shape[0], Cb.shape[0])])
    
    ycrcb = np.concatenate((Y, Cr, Cb))
    return ycrcb.tolist()

def inverse_zigzag_scan(ycrcb):
    def inverse_vecteur_zigzag(vector, shape):
        rows, cols = shape
        mat = np.array([[None] * cols for _ in range(rows)], dtype=np.float64)
        row, col = 0, 0
        direction = 1

        for i in range(rows * cols):
            mat[row][col] = vector[i]
            if direction == 1:
                if col == cols - 1:
                    row += 1
                    direction = -1
                elif row == 0:
                    col += 1
                    direction = -1
                else:
                    row -= 1
                    col += 1
            else:
                if row == rows - 1:
                    col += 1
                    direction = 1
                elif col == 0:
                    row += 1
                    direction = 1
                else:
                    row += 1
                    col -= 1

        return mat
    
    def diviser_liste(liste):
        if len(liste) < 96:
            return "La liste doit contenir au moins 95 éléments pour être divisée selon les spécifications."
        
        premiere_partie = liste[:64]
        deuxieme_partie = liste[64:80]
        troisieme_partie = liste[80:]

        return premiere_partie, deuxieme_partie, troisieme_partie
    
    # Séparer les composantes Y, Cr et Cb
    Y = diviser_liste(ycrcb)[0]
    Cr = diviser_liste(ycrcb)[1]
    Cb = diviser_liste(ycrcb)[2]
    
    Y = inverse_vecteur_zigzag(Y, (8, 8))
    Cr = inverse_vecteur_zigzag(Cr, (4, 4))
    Cb = inverse_vecteur_zigzag(Cb, (4, 4))
    
    Y = np.array(Y)
    Cr = np.array(Cr)
    Cb = np.array(Cb)
    
    # Arrondir les valeurs
    Y = np.round(Y)
    Cr = np.round(Cr)
    Cb = np.round(Cb)
    
    return [Y, Cr, Cb]

def calculate_mse(original_matrices, reconstructed_matrices):
    mse_total = 0
    
    # Parcourir les trois matrices
    for i in range(3):
        original_matrix = original_matrices[i]
        reconstructed_matrix = reconstructed_matrices[i]
        
        # Convertir les matrices en tableaux numpy
        original_array = np.array(original_matrix)
        reconstructed_array = np.array(reconstructed_matrix)
        
        # Calculer le MSE pour cette composante
        mse = np.mean((original_array - reconstructed_array) ** 2)
        mse_total += mse
    
    # Calculer le MSE moyen sur les trois composantes
    mse_average = mse_total / 3
    
    return mse_average

# Exemple d'utilisation
original_Y = np.random.rand(8, 8)  # Matrice Y originale de taille 8x8
original_Cr = np.random.rand(4, 4)  # Matrice Cr originale de taille 4x4
original_Cb = np.random.rand(4, 4)  # Matrice Cb originale de taille 4x4

# Créer la liste de trois matrices
original_matrices = [np.array(original_Y), np.array(original_Cr), np.array(original_Cb)]

# Appliquer la fonction zigzag_scan
zigzag_result = zigzag_scan(original_matrices)

# Appliquer la fonction inverse_zigzag_scan
reconstructed_matrices = inverse_zigzag_scan(zigzag_result)

# Calculer le MSE entre les matrices originales et les matrices reconstruites
mse = calculate_mse(original_matrices, reconstructed_matrices)

print("Mean Squared Error (MSE) entre les matrices originales et les matrices reconstruites:", mse)


Mean Squared Error (MSE) entre les matrices originales et les matrices reconstruites: 0.09443861545633885


In [5]:
import numpy as np

def mse(original_matrices, reconstructed_matrices):
    mse_total = 0
    
    # Parcourir les trois matrices
    for i in range(3):
        original_matrix = original_matrices[i]
        reconstructed_matrix = reconstructed_matrices[i]
        
        # Convertir les matrices en tableaux numpy
        original_array = np.array(original_matrix)
        reconstructed_array = np.array(reconstructed_matrix)
        
        # Calculer le MSE pour cette composante
        mse = np.mean((original_array - reconstructed_array) ** 2)
        mse_total += mse
    
    # Calculer le MSE moyen sur les trois composantes
    mse_average = mse_total / 3
    
    return mse_average

def reshape_and_average(matrix1):
    # Redimensionne la matrice en une matrice 4x4 en prenant la moyenne de chaque bloc 2x2
    reshaped = np.zeros((4, 4))
    for i in range(4):
        for j in range(4):
            # Calcule la moyenne des 4 valeurs dans le bloc 2x2
            block_mean = np.mean(matrix1[2*i:2*(i+1), 2*j:2*(j+1)])
            reshaped[i, j] = block_mean
    return reshaped

def quantization(matrix):
    # Récupération des composantes de l'image à partir de la liste
    luminance = matrix[0]  # Composante de luminance (Y)
    chrominance_U = matrix[1]
    chrominance_V = matrix[2]
    
    # Réplication des échantillons pour chaque bloc 2x2
    Cr = np.kron(chrominance_U, np.ones((2, 2)))
    Cb = np.kron(chrominance_V, np.ones((2, 2)))
    
    # Matrices de quantification de luminance et de chrominance
    luminance_quantization_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                              [12, 12, 14, 19, 26, 58, 60, 55],
                                              [14, 13, 16, 24, 40, 57, 69, 56],
                                              [14, 17, 22, 29, 51, 87, 80, 62],
                                              [18, 22, 37, 56, 68, 109, 103, 77],
                                              [24, 35, 55, 64, 81, 104, 113, 92],
                                              [49, 64, 78, 87, 103, 121, 120, 101],
                                              [72, 92, 95, 98, 112, 100, 103, 99]])

    chrominance_quantization_matrix = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                                [18, 21, 26, 66, 99, 99, 99, 99],
                                                [24, 26, 56, 99, 99, 99, 99, 99],
                                                [47, 66, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99]])
    
    # Applique les matrices de quantification à chaque composante
    luminance_quantized = np.round(luminance / luminance_quantization_matrix)
    chrominance_Cr_quantized = np.round(Cr / chrominance_quantization_matrix)
    chrominance_Cb_quantized = np.round(Cb / chrominance_quantization_matrix)
    
    chrominance_Cr_quantized = np.round(reshape_and_average(chrominance_Cr_quantized))
    chrominance_Cb_quantized = np.round(reshape_and_average(chrominance_Cb_quantized))
    
    quantized = [luminance_quantized, chrominance_Cr_quantized, chrominance_Cb_quantized]
    
    return quantized

def dequantization(matrix):
    # Récupération des composantes de l'image à partir de la liste
    L = matrix[0]  # Composante de luminance (Y)
    chrominance_Cr_quantized = matrix[1]
    chrominance_Cb_quantized = matrix[2]
    
    Cr = np.kron(chrominance_Cr_quantized, np.ones((2, 2)))
    Cb = np.kron(chrominance_Cb_quantized, np.ones((2, 2)))

    # Matrices de quantification de luminance et de chrominance
    luminance_quantization_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                              [12, 12, 14, 19, 26, 58, 60, 55],
                                              [14, 13, 16, 24, 40, 57, 69, 56],
                                              [14, 17, 22, 29, 51, 87, 80, 62],
                                              [18, 22, 37, 56, 68, 109, 103, 77],
                                              [24, 35, 55, 64, 81, 104, 113, 92],
                                              [49, 64, 78, 87, 103, 121, 120, 101],
                                              [72, 92, 95, 98, 112, 100, 103, 99]])

    chrominance_quantization_matrix = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                                [18, 21, 26, 66, 99, 99, 99, 99],
                                                [24, 26, 56, 99, 99, 99, 99, 99],
                                                [47, 66, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99]])

    # Multiplie chaque valeur quantifiée par sa correspondance dans la matrice de quantification respective
    luminance_dequantized = L * luminance_quantization_matrix
    chrominance_Cr_dequantized = Cr * chrominance_quantization_matrix
    chrominance_Cb_dequantized = Cb * chrominance_quantization_matrix

    # Restaure la résolution de chrominance U et V à 8x8
    chrominance_Cr_dequantized = reshape_and_average(chrominance_Cr_dequantized)
    chrominance_Cb_dequantized = reshape_and_average(chrominance_Cb_dequantized)

    # Retourne la liste des composantes [luminance, chrominance_U, chrominance_V]
    dequantized = [luminance_dequantized, chrominance_Cr_dequantized, chrominance_Cb_dequantized]

    return dequantized

original_Y = np.random.rand(8, 8)  # Matrice Y originale de taille 8x8
original_Cr = np.random.rand(4, 4)  # Matrice Cr originale de taille 4x4
original_Cb = np.random.rand(4, 4)  # Matrice Cb originale de taille 4x4

# Créer la liste de trois matrices
original_matrices = [np.array(original_Y), np.array(original_Cr), np.array(original_Cb)]

# Appliquer la fonction zigzag_scan
zigzag_result = quantization(original_matrices)

# Appliquer la fonction inverse_zigzag_scan
reconstructed_matrices = dequantization(zigzag_result)

# Calculer le MSE entre les matrices originales et les matrices reconstruites
mse = mse(original_matrices, reconstructed_matrices)

print("Mean Squared Error (MSE) entre les matrices originales et les matrices reconstruites:", mse)


Mean Squared Error (MSE) entre les matrices originales et les matrices reconstruites: 0.3260134788711962


In [3]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct

def mse(image1, image2):
    # Convertir les images en tableaux numpy
    img1_np = np.array(image1)
    img2_np = np.array(image2)
    
    # Calculer la Mean Squared Error (MSE)
    mse = np.mean((img1_np - img2_np) ** 2)
    
    return mse

def apply_dct(matrix):
    # Appliquer la DCT à chaque matrice
    dct_matrice0 = dct(dct(matrix[0].T, norm='ortho').T, norm='ortho')
    dct_matrice1 = dct(dct(matrix[1].T, norm='ortho').T, norm='ortho')
    dct_matrice2 = dct(dct(matrix[2].T, norm='ortho').T, norm='ortho')

    dct_matrix = [dct_matrice0, dct_matrice1, dct_matrice2]
    
    return dct_matrix

def inverse_dct(dct_matrix):
    # Appliquer l'inverse de la DCT à chaque matrice
    inverse_dct_matrice0 = idct(idct(dct_matrix[0].T, norm='ortho').T, norm='ortho')
    inverse_dct_matrice1 = idct(idct(dct_matrix[1].T, norm='ortho').T, norm='ortho')
    inverse_dct_matrice2 = idct(idct(dct_matrix[2].T, norm='ortho').T, norm='ortho')

    inverse_dct_matrix = [inverse_dct_matrice0, inverse_dct_matrice1, inverse_dct_matrice2]
    
    return inverse_dct_matrix

# Exemple d'utilisation :
# img_orig est l'image d'origine
img_orig = Image.open("eren_rgb.jpg")  # Remplacez "img_orig.jpg" par le chemin de votre image

import numpy as np

# Création d'une matrice tridimensionnelle avec des matrices 8x8
matrice_tridimensionnelle = np.zeros((3, 8, 8))

# Remplissage de chaque matrice avec des valeurs différentes (à titre d'exemple)
matrice_tridimensionnelle[0] = np.ones((8, 8))  # Remplir la première dimension avec des 1
matrice_tridimensionnelle[1] = np.eye(8)        # Remplir la deuxième dimension avec une matrice identité
matrice_tridimensionnelle[2] = np.random.randint(0, 10, (8, 8))  # Remplir la troisième dimension avec des valeurs aléatoires entre 0 et 9

# Affichage de la matrice tridimensionnelle
print(matrice_tridimensionnelle)


img_orig_np = np.array(matrice_tridimensionnelle)
#img_orig_np = np.array(img_orig, dtype = np.uint8)

dct_matrix = apply_dct(img_orig_np)

# Reconstruction de l'image après application de la DCT et son inverse
img_reconstruite = inverse_dct(dct_matrix)

# Calcul de la MSE entre l'image d'origine et l'image reconstruite
erreur_mse = mse(img_orig_np, img_reconstruite)
print("Mean Squared Error entre l'image d'origine et l'image reconstruite:", erreur_mse)

[[[1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1.]]

 [[8. 3. 5. 5. 8. 6. 4. 2.]
  [9. 6. 5. 6. 4. 6. 3. 1.]
  [8. 7. 3. 2. 8. 7. 4. 9.]
  [2. 8. 6. 2. 9. 7. 4. 9.]
  [1. 5. 6. 8. 6. 6. 4. 7.]
  [2. 6. 0. 0. 5. 8. 8. 8.]
  [6. 6. 5. 2. 8. 6. 0. 7.]
  [2. 6. 2. 4. 0. 8. 8. 3.]]]
Mean Squared Error entre l'image d'origine et l'image reconstruite: 1.400909404485123e-30


In [10]:
import numpy as np
from PIL import Image

def mse(image1, image2):
    # Convertir les images en tableaux numpy
    img1_np = np.array(image1)
    img2_np = np.array(image2)
    
    # Calculer la Mean Squared Error (MSE)
    mse = np.mean((img1_np - img2_np) ** 2)
    
    return mse

def sous_echantillonnage(transcol, SSV, SSH):
    # Convertir l'image en tableau NumPy
    transcol_np = np.array(transcol)
    
    # Filtrage des canaux de chrominance
    crf = transcol_np[:,:,1]
    cbf = transcol_np[:,:,2]
    
    # Sous-échantillonnage des canaux de chrominance
    crsub = crf[::SSV, ::SSH]
    cbsub = cbf[::SSV, ::SSH]
    
    # Créer la liste contenant tous les canaux
    imSub = [np.array(transcol_np[:,:,0]), np.array(crsub), np.array(cbsub)]
    
    return imSub

def inverse_sous_echantillonnage(imSub, SSV, SSH):
    # Récupérer les canaux sous-échantillonnés
    lum, _, _ = imSub
    
    # Initialiser des tableaux pour les canaux complets
    H, W = imSub[1].shape
    crf = np.zeros((H * SSH, W * SSV))
    cbf = np.zeros((H * SSH, W * SSV))
    
    # Créer l'image complète
    image_complete = np.zeros((H * SSV, W * SSH, 3), dtype=np.uint8)
    image_complete[:,:,0] = lum
    image_complete[:,:,1] = np.repeat(np.repeat(imSub[1], SSH, axis=0), SSV, axis=1)
    image_complete[:,:,2] = np.repeat(np.repeat(imSub[2], SSH, axis=0), SSV, axis=1)
    
    # Convertir le tableau numpy en image PIL
    transcol = Image.fromarray(image_complete)
    
    return transcol

# Exemple d'utilisation :
# transcol est l'image d'entrée
transcol = Image.open("eren_rgb.png")  # Remplacez "transcol.jpg" par le chemin de votre image
SSV = 2  # Facteur de sous-échantillonnage vertical
SSH = 2  # Facteur de sous-échantillonnage horizontal

# Sous-échantillonnage de l'image d'entrée
imSub = sous_echantillonnage(transcol, SSV, SSH)

# Reconstruction de l'image après sous-échantillonnage
transcol_reconstituee = inverse_sous_echantillonnage(imSub, SSV, SSH)

# Calcul de la MSE entre l'image d'entrée et l'image reconstruite
erreur_mse = mse(np.array(transcol), np.array(transcol_reconstituee))
print("Mean Squared Error entre l'image d'entrée et l'image reconstruite:", erreur_mse)


Mean Squared Error entre l'image d'entrée et l'image reconstruite: 4.923622730242108


In [6]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def subdivision(image, taille_bloc):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    # Obtenir la largeur et la hauteur de l'image
    largeur, hauteur = image.size

    # Calculer le nombre de blocs en largeur et en hauteur
    blocs_largeur = (largeur + taille_bloc - 1) // taille_bloc
    blocs_hauteur = (hauteur + taille_bloc - 1) // taille_bloc

    # Créer une nouvelle image avec les dimensions ajustées
    nouvelle_largeur = blocs_largeur * taille_bloc
    nouvelle_hauteur = blocs_hauteur * taille_bloc
    nouvelle_image = Image.new(image.mode, (nouvelle_largeur, nouvelle_hauteur), color='black')
    nouvelle_image.paste(image, (0, 0))

    blocs = []
    # Parcourir chaque bloc
    for y in range(blocs_hauteur):
        for x in range(blocs_largeur):
            # Obtenir les coordonnées du coin supérieur gauche du bloc
            coin_sup_gauche_x = x * taille_bloc
            coin_sup_gauche_y = y * taille_bloc
            # Obtenir les coordonnées du coin inférieur droit du bloc
            coin_inf_droit_x = (x + 1) * taille_bloc
            coin_inf_droit_y = (y + 1) * taille_bloc
            # Extraire le bloc de l'image
            bloc = nouvelle_image.crop((coin_sup_gauche_x, coin_sup_gauche_y, coin_inf_droit_x, coin_inf_droit_y))
            blocs.append(bloc)
            # Afficher le bloc
            plt.subplot(blocs_hauteur, blocs_largeur, y * blocs_largeur + x + 1)
            plt.imshow(bloc)
            plt.axis('off')

    # Afficher les blocs
    plt.figure(figsize=(20, 10))
    plt.show()

    return blocs, (largeur, hauteur)

def reconstituer_image(blocs, taille_bloc, dimensions_originales):
    largeur_orig, hauteur_orig = dimensions_originales
    
    # Créer une nouvelle image vide avec les dimensions originales
    image_reconstituee = np.zeros((hauteur_orig, largeur_orig, 3), dtype=np.uint8)
    
    # Parcourir chaque bloc et le placer à la position correspondante dans l'image reconstituée
    idx = 0
    for y in range(0, hauteur_orig, taille_bloc):
        for x in range(0, largeur_orig, taille_bloc):
            # Extraire le bloc de l'image reconstituée
            bloc = blocs[idx]
            idx += 1
            # Convertir le bloc en tableau numpy
            bloc_np = np.array(bloc, dtype=np.uint8)
            
            if bloc_np.shape[2] > bloc_np.shape[0]:
                bloc_np = np.transpose(bloc_np, (2, 1, 0))
            # Calculer les coordonnées de collage du bloc dans l'image reconstituée
            x_end = min(x + taille_bloc, largeur_orig)
            y_end = min(y + taille_bloc, hauteur_orig)
            # Coller le bloc dans l'image reconstituée
            image_reconstituee[y:y_end, x:x_end] = bloc_np[:y_end-y, :x_end-x]

    # Créer une image PIL à partir du tableau numpy
    image_reconstituee_pil = Image.fromarray(image_reconstituee)
    
    return image_reconstituee_pil

def mse(image1, image2):
    # Convertir les images en tableaux numpy
    img1_np = np.array(image1)
    img2_np = np.array(image2)
    
    # Calculer la Mean Squared Error (MSE)
    mse = np.mean((img1_np - img2_np) ** 2)
    
    return mse

# Exemple d'utilisation:
image = Image.open("eren_rgb.jpeg")  # Remplacez "example.jpg" par le chemin de votre image
blocs, dimensions_originales = subdivision(image, 8)
image_reconstituee = reconstituer_image(blocs, 8, dimensions_originales)
erreur = mse(np.array(image), np.array(image_reconstituee))
print("Mean Squared Error entre l'image originale et l'image reconstituée:", erreur)


/home/issa-sangare/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
import cv2
import numpy as np
from PIL import Image

def rgb_to_ycrcb(rgb_image):
    ycrcb_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2YCrCb)
    return ycrcb_image

def ycrcb_to_rgb(ycrcb_image):
    rgb_image = cv2.cvtColor(ycrcb_image, cv2.COLOR_YCrCb2RGB)
    return rgb_image

def mse(image1, image2):
    squared_diff = (image1 - image2) ** 2
    mean_squared_diff = np.mean(squared_diff)
    return mean_squared_diff

def main():
    # Charger l'image
    img_path = "eren_rgb.jpeg"
    rgb_image = np.array(Image.open(img_path))

    # Convertir en YCbCr
    ycrcb_image = rgb_to_ycrcb(rgb_image)

    # Convertir en RGB à nouveau
    rgb_restored_image = ycrcb_to_rgb(ycrcb_image)

    # Calculer le MSE entre les deux images
    mse_value = mse(rgb_image, rgb_restored_image)

    print("Mean Squared Error (MSE) entre les deux images : ", mse_value)

if __name__ == "__main__":
    main()

Mean Squared Error (MSE) entre les deux images :  0.033361731384330254


In [3]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image

def compression_gif():
    # Sélectionner le fichier image à compresser
    file_path = filedialog.askopenfilename()
    if file_path:
        # Ouvrir l'image
        with open(file_path, 'rb') as img_file:
            img_data = img_file.read()
        
        # Convertir les données de l'image en une chaîne de texte
        text_data = img_data.hex()
        
        # Écrire la chaîne de texte dans un fichier avec le même nom, mais l'extension .txt
        text_path = file_path.rsplit('.', 1)[0] + '.txt'
        with open(text_path, 'w') as text_file:
            text_file.write(text_data)
        
        print("L'image a été compressée avec succès en format texte :", text_path)

def decompression_gif():
    # Sélectionner le fichier texte à décompresser
    file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
    if file_path:
        # Lire le contenu du fichier texte
        with open(file_path, 'r') as text_file:
            text_data = text_file.read()
        
        # Convertir la chaîne de texte en données binaires
        img_data = bytes.fromhex(text_data)
        
        # Enregistrer les données binaires dans un fichier image avec le même nom, mais l'extension .png
        image_path = file_path.rsplit('.', 1)[0] + '_decompressed.png'
        with open(image_path, 'wb') as img_file:
            img_file.write(img_data)
        
        print("L'image a été décompressée avec succès :", image_path)

# Créer une fenêtre Tkinter
root = tk.Tk()
root.title("Compression / Décompression d'images")

# Bouton pour la compression
compress_button = tk.Button(root, text="Compresser une image", command=compression_gif)
compress_button.pack(pady=10)

# Bouton pour la décompression
decompress_button = tk.Button(root, text="Décompresser un fichier texte", command=decompression_gif)
decompress_button.pack(pady=10)

# Lancer la boucle principale Tkinter
root.mainloop()

L'image a été compressée avec succès en format texte : /home/issa-sangare/Documents/TDM/Image_Compression/eren_rgb.txt
L'image a été décompressée avec succès : /home/issa-sangare/Documents/TDM/Image_Compression/eren_rgb_decompressed.png


In [ ]:
def data_gif(img_path, palette):
  image = io.imread(img_path)
  color_palette, _ = create_color_palette(image, palette)
  alphabet = [str(i) for i in range(len(color_palette))]
  mapper, _ = map_to_palette(image, color_palette)
  code, _ = encoding_lzw(mapper, alphabet)
  
  return code

In [ ]:
def header_gif(img_path):
  

In [ ]:
def create_gif_header(image_size):
    """
    Crée l'entête GIF à partir de la forme de l'image.
    """
    width, height = image_size
    header = bytearray()
    
    # Signature GIF
    header.extend(b'GIF89a')
    
    # Largeur et hauteur de l'image
    header.extend(width.to_bytes(2, byteorder='little'))
    header.extend(height.to_bytes(2, byteorder='little'))
    
    # Flags de palettes de couleur
    flags = 0b10000000  # Utiliser une palette de couleur globale
    flags |= 0b111  # Couleur de la palette résolution à 8 bits
    header.append(flags)
    
    # Couleur d'arrière-plan (index de la palette)
    header.append(0)
    
    # Ratio d'aspect
    header.append(0)  # Par défaut
    
    # Palette de couleur globale (par défaut)
    # Vous pouvez ajouter une palette de couleurs ici si nécessaire
    
    return header

# Exemple d'utilisation
image_path = "eren_rgb.jpg"
image = Image.open(image_path)
gif_header = create_gif_header(image.size)

In [ ]:
def generate_image_header(format_type, width, height, color_depth):
    if format_type == "JPEG":
        # En-tête JPEG
        header = b"\xFF\xD8"  # Marqueur d'entête SOI (Start of Image)
        header += b'\xFF\xE0'  # Marqueur APP0 (JFIF)
        header += b'JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00'
        return header
    
    elif format_type == "GIF":
        # En-tête GIF
        header = b"GIF89a"  # En-tête GIF89a
        logical_screen_descriptor = width.to_bytes(2, byteorder='little') + height.to_bytes(2, byteorder='little')
        header += logical_screen_descriptor  # Descripteur d'écran logique
        color_table = b'\x00' * 6  # Table de couleurs (exemple de table vide)
        header += color_table
        return header
    
    elif format_type == "PNG":
        # En-tête PNG
        header = b"\x89PNG\r\n\x1a\n"  # En-tête PNG
        header += b"\x00\x00\x00\x0D"  # Longueur de l'en-tête IHDR
        header += b"IHDR"  # Signature IHDR
        header += width.to_bytes(4, byteorder='big')  # Largeur de l'image
        header += height.to_bytes(4, byteorder='big')  # Hauteur de l'image
        header += bytes([color_depth])  # Profondeur de couleur
        header += bytes([0])  # Type de compression (non compressé dans cet exemple)
        header += bytes([0])  # Type de filtrage (non filtré dans cet exemple)
        header += bytes([0])  # Méthode d'interprétation (non interlacé dans cet exemple)
        header += b"\x00\x00\x00\x00"  # CRC (non utilisé dans cet exemple)
        palette = b''  # Palette (exemple de palette vide)
        header += palette
        return header
    
    elif format_type == "BMP":
        # En-tête BMP
        header = b"BM"  # En-tête BMP
        file_size = 14 + 40 + (width * height * (color_depth // 8))  # Taille totale du fichier
        header += file_size.to_bytes(4, byteorder='little')  # Taille totale du fichier
        header += b"\x00\x00"  # Réservé
        header += b"\x00\x00"  # Réservé
        header += (14 + 40).to_bytes(4, byteorder='little')  # Début des données d'image
        return header
    
    elif format_type == "TIFF":
        # En-tête TIFF
        header = b"II*\x00"  # En-tête TIFF (Little-endian)
        header += b"\x2A\x00"  # Numéro de version
        offset = 8  # Offset de l'en-tête IFD
        header += offset.to_bytes(4, byteorder='little')  # Offset de l'en-tête IFD
        return header
    
    else:
        raise ValueError("Format d'image non pris en charge")

# Exemple d'utilisation
jpeg_header = generate_image_header("JPEG", 800, 600, 24)
gif_header = generate_image_header("GIF", 800, 600, 8)
png_header = generate_image_header("PNG", 800, 600, 24)
bmp_header = generate_image_header("BMP", 800, 600, 24)
tiff_header = generate_image_header("TIFF", 800, 600, 24)

# Affichage des en-têtes générés
print("JPEG Header:", jpeg_header)
print("GIF Header:", gif_header)
print("PNG Header:", png_header)
print("BMP Header:", bmp_header)
print("TIFF Header:", tiff_header)